In [26]:
import os
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_neo4j import Neo4jChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# --- 1. CONFIGURATION ---
CHROMA_PATH_PUBLIC = r"C:\Users\shreyas\Aerothon\expirements\db_root\public\Knowledge_vectors"
CHROMA_PATH_SECURE = r"C:\Users\shreyas\Aerothon\expirements\secure_DB\secure\Knowledge_vectors"

NEO4J_PUBLIC_URI = "neo4j://localhost:7687"
NEO4J_SECURE_URI = "neo4j://localhost:7688"
NEO4J_AUTH = ("neo4j", "password")

OLLAMA_BASE_URL = "http://127.0.0.1:11434"
MODEL_NAME = "llama3.1:8b-instruct-q4_K_M"

# --- 2. INITIALIZE ---
llm = ChatOllama(model=MODEL_NAME, temperature=0, base_url=OLLAMA_BASE_URL)
embeddings = OllamaEmbeddings(model="nomic-embed-text", base_url=OLLAMA_BASE_URL)

vectorstore_public = Chroma(persist_directory=CHROMA_PATH_PUBLIC, embedding_function=embeddings)
vectorstore_secure = Chroma(persist_directory=CHROMA_PATH_SECURE, embedding_function=embeddings)

# --- 3. STRICT RETRIEVAL ---
def get_combined_context(query: str):
    # Retrieve chunks from both isolated vector stores
    public_docs = vectorstore_public.as_retriever(search_kwargs={"k": 5}).invoke(query)
    secure_docs = vectorstore_secure.as_retriever(search_kwargs={"k": 5}).invoke(query)
    
    # Combined text without invented "Archive" labels to stay text-grounded
    combined_text = "\n\n".join([d.page_content for d in (public_docs + secure_docs)])
    return combined_text, public_docs + secure_docs

# --- 4. CORRECTED CHAT LOGIC ---
def experimental_chat(user_input: str, session_id: str, use_secure_history: bool = True):
    # Select the isolated Neo4j container for history
    history_uri = NEO4J_SECURE_URI if use_secure_history else NEO4J_PUBLIC_URI
    
    history = Neo4jChatMessageHistory(
        url=history_uri, username=NEO4J_AUTH[0], password=NEO4J_AUTH[1], session_id=session_id
    )

    # 1. Retrieve Context
    context_text, all_docs = get_combined_context(user_input)

    # 2. Updated Prompt: Enforcing Strict Integrity
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a professional Document Analyst. 
        
        CRITICAL RULES:
        1. Answer ONLY using the provided Context. 
        2. Do NOT introduce external information (phone numbers, emails, or names) not found in the text.
        3. If the text provides multiple options, you MUST mention all of them.
        4. Use the exact terminology found in the document.
        5. If the answer is not in the context, say: 'Information not found in provided materials.'"""),
        MessagesPlaceholder("chat_history"),
        ("human", "Context:\n{context}\n\nQuestion: {input}"),
    ])

    # 3. Execution
    chain = prompt | llm
    response = chain.invoke({
        "input": user_input,
        "chat_history": history.messages,
        "context": context_text
    })

    # 4. Persistence
    history.add_user_message(user_input)
    history.add_ai_message(response.content)

    return response.content, all_docs



In [27]:
if __name__ == "__main__":
    query = "What is meant by undue advantage, and when does it arise?"
    answer, docs = experimental_chat(query, "exp_user_02")
    print(f"AI RESPONSE:\n{answer}")

AI RESPONSE:
According to the context, Undue Advantage means any information or assistance that a candidate receives from any source during the examination which gives him an unfair advantage over other candidates.

Undue advantage arises when a candidate uses any unauthorized material or device during the examination.
